# Overview

This notebook aims to follow a clustering and scoring approach for phishing email detection.

## High level architecture:

* Pre-process emails (to remove garbage characters and such) and split them into test / train and validation sets
* Pre-process the emails with sentence transformer
* Visualise the email embeddings
* Cluster the email embeddings
* Give a phishing score for each cluster based on how many emails in the cluster are phishing and how many are safe. As well calculate a mean vector for each cluster.
* Test by having the validation and test email datasets compared based on cosine similarity to the cluster medoid vector
* Give the test / validation email a score based on their distance from the nearest cluster
* Calculate an "accuracy" based on a scoring threshold

This notebook can be accessed on google colab here:
https://colab.research.google.com/drive/1Ctsy9GCyiSvjFEXxLr6vNKer0-oERwX9?usp=sharing

## Download the dataset files

In [ ]:
!git clone https://github.com/jk5279/Phishnet.git

Cloning into 'Phishnet'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 218 (delta 66), reused 191 (delta 46), pack-reused 0 (from 0)
Receiving objects: 100% (218/218), 9.03 MiB | 12.61 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Error downloading object: archive/Emails_generated_by_Abhay.csv (c267682): Smudge error: Error downloading archive/Emails_generated_by_Abhay.csv (c26768289b95be69efdac2382fb598b12aad2417441cddea229eb0f5444ba272): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/Phishnet/.git/lfs/logs/20251122T052301.321398218.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: archive/Emails_generated_by_Abhay.csv: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and re

In [1]:
!wget "https://utoronto-my.sharepoint.com/:u:/g/personal/marc_bishara_mail_utoronto_ca/ET-R_B9KUhNPkdJdwO6TSlkB7kEUdW_VL6I_BnLBTZOP8w?download=1" -O /content/Phishnet.zip

--2025-11-23 01:32:16--  https://utoronto-my.sharepoint.com/:u:/g/personal/marc_bishara_mail_utoronto_ca/ET-R_B9KUhNPkdJdwO6TSlkB7kEUdW_VL6I_BnLBTZOP8w?download=1
Resolving utoronto-my.sharepoint.com (utoronto-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to utoronto-my.sharepoint.com (utoronto-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/marc_bishara_mail_utoronto_ca/Documents/Phishnet.zip?ga=1 [following]
--2025-11-23 01:32:16--  https://utoronto-my.sharepoint.com/personal/marc_bishara_mail_utoronto_ca/Documents/Phishnet.zip?ga=1
Reusing existing connection to utoronto-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 2375019548 (2.2G) [application/x-zip-compressed]
Saving to: ‘/content/Phishnet.zip’

/content/Phishnet.z 100%[===================>]   2.21G  82.4MB/s    in 26s     

2025-11-23 01:32:43 (86.7 MB/s) - ‘/content/Phishnet.zip’ save

In [2]:
!unzip -q /content/Phishnet.zip -d /content/

## Preprocess the emails and split them

In [3]:
# Complete Data Preparation Code

import os
import sys
import glob
import re
import quopri
import multiprocessing
from typing import List, Dict, Optional
import warnings
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from bs4 import MarkupResemblesLocatorWarning
from tqdm.contrib.concurrent import process_map
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Configuration
RAW_DATA_DIR = "/content/Phishnet/datasets/raw - DO NOT OVERWRITE"
TEST_DATASETS_DIR = "/content/Phishnet/datasets/raw - DO NOT OVERWRITE/test_datasets"
OUTPUT_DIR = "outputs"

COLUMN_CONFIG = {
    "text_columns": ["Email Text", "Text", "text", "body"],
    "label_columns": ["Email Type", "Class", "label"],
}

LABEL_MAP = {
    "Phishing Email": 1, "1.0": 1, "1": 1, "spam": 1,
    "Safe Email": 0, "0.0": 0, "0": 0, "ham": 0,
}

#This list will contain the file names from which the emails came. The list index will be used to track which email text comes form which file
dataFileNames = []

NUM_WORKERS = multiprocessing.cpu_count()
SAMPLE_SIZE = 1000

# Helper Functions
def set_max_csv_field_size_limit():
    """Set the CSV field size limit as high as possible."""
    import csv
    max_int = sys.maxsize
    while True:
        try:
            csv.field_size_limit(max_int)
            break
        except OverflowError:
            max_int = int(max_int / 10)

def is_git_lfs_pointer(df: pd.DataFrame) -> bool:
    """Check if DataFrame appears to be a Git LFS pointer file."""
    if df is None or df.empty:
        return False
    columns = list(df.columns)
    has_lfs_column = any('version https://git-lfs.github.com/spec/v1' in str(col) for col in columns)
    if has_lfs_column and len(df) > 0:
        first_val = str(df.iloc[0, 0]) if len(df) > 0 else ""
        has_oid = 'oid sha256:' in first_val
        return has_oid
    return has_lfs_column

def safe_read_csv(path: str) -> Optional[pd.DataFrame]:
    """Read a CSV using robust fallbacks for different encodings and formats."""
    try:
        df = pd.read_csv(path, engine="c")
        if is_git_lfs_pointer(df):
            return None
        return df
    except Exception:
        pass
    try:
        df = pd.read_csv(path, engine="c", encoding="latin1", on_bad_lines="skip")
        if is_git_lfs_pointer(df):
            return None
        return df
    except Exception:
        pass
    try:
        df = pd.read_csv(path, engine="python")
        if is_git_lfs_pointer(df):
            return None
        return df
    except Exception:
        return None

def consolidate_column(df: pd.DataFrame, possible_cols: List[str]) -> pd.Series:
    """Consolidate multiple possible column names into a single series."""
    result = None
    for col in possible_cols:
        if col in df.columns:
            if result is None:
                result = df[col]
            else:
                result = result.fillna(df[col])
    if result is None:
        raise ValueError(f"None of the expected columns found: {possible_cols}")
    return result.astype(str).replace("nan", "").fillna("")

def clean_email_text(text: object) -> str:
    """Clean email text by removing HTML, normalizing URLs/emails."""
    if not isinstance(text, str):
        return ""
    # Decode quoted-printable
    try:
        text_bytes = text.encode("latin-1", errors="ignore")
        decoded_bytes = quopri.decodestring(text_bytes)
        text = decoded_bytes.decode("utf-8", errors="ignore")
    except Exception:
        pass
    # Strip HTML
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
    # Lowercase
    text = text.lower()
    # Replace URLs
    text = re.sub(r"(https?://\S+|www\.\S+)", "[url]", text)
    # Replace emails
    text = re.sub(r"\b[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}\b", "[email]", text)
    # Keep alnum, space, and []
    text = re.sub(r"[^a-z0-9\s\[\]]", "", text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

def combine_dataset_files() -> pd.DataFrame:
    """Combine all CSV files from Dataset/raw, excluding test_datasets."""
    set_max_csv_field_size_limit()
    if not os.path.isdir(RAW_DATA_DIR):
        raise FileNotFoundError(f"Directory not found: {RAW_DATA_DIR}")
    pattern = os.path.join(RAW_DATA_DIR, "**", "*.csv")
    all_files = glob.glob(pattern, recursive=True)
    files = [f for f in all_files if TEST_DATASETS_DIR not in f]
    if not files:
        raise FileNotFoundError(f"No CSV files found in: {RAW_DATA_DIR}")
    dfs: List[pd.DataFrame] = []
    git_lfs_count = 0
    for f in files:
        df = safe_read_csv(f)
        if df is not None and not df.empty:
            if is_git_lfs_pointer(df):
                git_lfs_count += 1
                continue
            df["source_file"] = os.path.basename(f)
            relative_dir = os.path.relpath(os.path.dirname(f), RAW_DATA_DIR)
            if relative_dir == ".":
                df["source_name"] = os.path.basename(os.path.normpath(RAW_DATA_DIR))
            else:
                df["source_name"] = relative_dir.split(os.sep)[0]
            dfs.append(df)
    if not dfs:
        error_msg = f"\n{'='*60}\nERROR: No valid CSV data files could be read.\n"
        if git_lfs_count > 0:
            error_msg += f"Found {git_lfs_count} Git LFS pointer files.\n\n"
            error_msg += "SOLUTION: Download the actual data files using Git LFS:\n"
            error_msg += "  1. Make sure Git LFS is installed: git lfs install\n"
            error_msg += "  2. Pull the actual files: git lfs pull\n"
            error_msg += "  3. Run this script again\n"
        error_msg += "="*60 + "\n"
        raise RuntimeError(error_msg)
    combined = pd.concat(dfs, ignore_index=True)
    return combined

def process_to_master_format(df: pd.DataFrame) -> pd.DataFrame:
    """Consolidate text/label columns and standardize labels."""
    master_text = consolidate_column(df, COLUMN_CONFIG["text_columns"])
    master_label_src = consolidate_column(df, COLUMN_CONFIG["label_columns"])
    normalized_map = {str(k).lower().strip(): v for k, v in LABEL_MAP.items()}
    normalized_labels = master_label_src.astype(str).str.lower().str.strip()
    master_label = normalized_labels.map(normalized_map)
    final_df = pd.DataFrame({"text": master_text, "label": master_label, "source_name": df["source_name"], "source_file": df["source_file"]})
    final_df = final_df.dropna(subset=["text", "label", "source_name", "source_file"])
    if len(final_df) == 0:
        raise RuntimeError("Master dataset empty after cleaning.")
    final_df["label"] = final_df["label"].astype(int)
    return final_df

def clean_dataset_text(df: pd.DataFrame) -> pd.DataFrame:
    """Apply text cleaning to the dataset."""
    df = df.copy()
    df["text"] = df["text"].fillna("")
    results = process_map(
        clean_email_text, df["text"],
        max_workers=NUM_WORKERS, chunksize=500, desc="Cleaning Emails"
    )
    df["text"] = results
    df = df[df["text"].str.strip().str.len() > 0]
    return df

def sample_dataset(df: pd.DataFrame, n_samples: int = SAMPLE_SIZE) -> pd.DataFrame:
    """Sample n_samples from dataset, stratified by label if possible."""
    if len(df) <= n_samples:
        return df
    try:
        sampled, _ = train_test_split(
            df, train_size=n_samples, stratify=df["label"], random_state=42
        )
        return sampled.reset_index(drop=True)
    except ValueError:
        return df.sample(n=n_samples, random_state=42).reset_index(drop=True)

def load_test_datasets() -> pd.DataFrame:
    """Load all CSV files from test_datasets directory and process them."""
    if not os.path.isdir(TEST_DATASETS_DIR):
        print(f"Warning: Test datasets directory not found: {TEST_DATASETS_DIR}")
        return pd.DataFrame(columns=["text", "label", "source_file", "source_name"])

    test_dfs = []
    pattern = os.path.join(TEST_DATASETS_DIR, "*.csv")
    files = glob.glob(pattern)

    if not files:
        print(f"Warning: No CSV files found in test datasets directory: {TEST_DATASETS_DIR}")
        return pd.DataFrame(columns=["text", "label", "source_file", "source_name"])

    git_lfs_count = 0
    for f in files:
        df = safe_read_csv(f)
        if df is not None and not df.empty:
            if is_git_lfs_pointer(df):
                git_lfs_count += 1
                print(f"Warning: Skipping Git LFS pointer file: {os.path.basename(f)}")
                continue
            # Attempt to consolidate columns based on COLUMN_CONFIG
            try:

                df["source_file"] = os.path.basename(f)
                relative_dir = os.path.relpath(os.path.dirname(f), RAW_DATA_DIR)
                if relative_dir == ".":
                  df["source_name"] =  os.path.basename(os.path.normpath(RAW_DATA_DIR))
                else:
                  df["source_name"] = relative_dir.split(os.sep)[0]

                processed_df = process_to_master_format(df)
                test_dfs.append(processed_df)
            except ValueError as e:
                print(f"Warning: Could not process file {os.path.basename(f)}: {e}")
                print("         Skipping this file.")
                continue
        elif df is None:
             print(f"Warning: Could not read file: {os.path.basename(f)}")
        elif df.empty:
             print(f"Warning: Skipping empty file: {os.path.basename(f)}")


    if not test_dfs:
        error_msg = f"\n{'='*60}\nERROR: No valid test CSV data files could be read.\n"
        if git_lfs_count > 0:
            error_msg += f"Found {git_lfs_count} Git LFS pointer files. Use 'git lfs pull' to download.\n"
        error_msg += "="*60 + "\n"
        print(error_msg) # Print error instead of raising to allow pipeline to continue
        return pd.DataFrame(columns=["text", "label"])


    combined_test = pd.concat(test_dfs, ignore_index=True)
    combined_test["text"] = combined_test["text"].fillna("")

    # Clean text for all test datasets
    results = process_map(
        clean_email_text, combined_test["text"],
        max_workers=NUM_WORKERS, chunksize=500, desc="Cleaning Test Emails"
    )
    combined_test["text"] = results
    combined_test = combined_test[combined_test["text"].str.strip().str.len() > 0]

    # Ensure labels are integers
    combined_test['label'] = combined_test['label'].astype(int)

    return combined_test

def create_train_val_test_splits(sampled_df: pd.DataFrame, test_datasets_df: pd.DataFrame) -> tuple:
    """Combine datasets and create train/val/test splits."""
    combined_df = pd.concat([sampled_df, test_datasets_df], ignore_index=True)
    train_df, temp_df = train_test_split(
        combined_df, test_size=0.3, stratify=combined_df["label"], random_state=42
    )
    val_df, test_df = train_test_split(
        temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42
    )
    return train_df, val_df, test_df, combined_df

print("✓ All data preparation functions loaded (complete implementation from 00_data_preparation.py)")
print("\nMain pipeline steps:")
print("1. Combine CSV files from Dataset/raw (excluding test_datasets)")
print("2. Consolidate text/label columns and standardize labels (0=Safe, 1=Phishing)")
print("3. Apply text cleaning (clean_email_text function)")
print("4. Sample approximately 1000 samples (stratified by label)")
print("5. Load test_datasets (all CSV files in the test_datasets directory)")
print("6. Create train/validation/test splits (70/15/15)")
print("\nOutputs: report/outputs/train.csv, val.csv, test.csv, combined_dataset_final.csv")

✓ All data preparation functions loaded (complete implementation from 00_data_preparation.py)

Main pipeline steps:
1. Combine CSV files from Dataset/raw (excluding test_datasets)
2. Consolidate text/label columns and standardize labels (0=Safe, 1=Phishing)
3. Apply text cleaning (clean_email_text function)
4. Sample approximately 1000 samples (stratified by label)
5. Load test_datasets (all CSV files in the test_datasets directory)
6. Create train/validation/test splits (70/15/15)

Outputs: report/outputs/train.csv, val.csv, test.csv, combined_dataset_final.csv


In [4]:
# Execute data preparation if outputs don't exist
print("\n" + "=" * 60)
print("Checking if data preparation outputs exist...")
print("=" * 60)

os.makedirs(OUTPUT_DIR, exist_ok=True)
train_output = os.path.join(OUTPUT_DIR, "train.csv")
val_output = os.path.join(OUTPUT_DIR, "val.csv")
test_output = os.path.join(OUTPUT_DIR, "test.csv")
combined_output = os.path.join(OUTPUT_DIR, "combined_dataset_final.csv")

if not (os.path.exists(train_output) and os.path.exists(val_output) and os.path.exists(test_output)):
    print("\nData preparation outputs not found. Running data preparation pipeline...")
    print("This may take a few minutes...")

    # Step 1: Combine dataset files
    print("\n[1/6] Combining CSV files from Dataset/raw...")
    combined_raw = combine_dataset_files()
    print(f"  ✓ Combined {len(combined_raw):,} raw samples")

    # Step 2: Process to master format
    print("\n[2/6] Processing to master format...")
    master_df = process_to_master_format(combined_raw)
    print(f"  ✓ Master format: {len(master_df):,} samples")

    # Step 3: Clean text
    print("\n[3/6] Cleaning email text...")
    cleaned_df = clean_dataset_text(master_df)
    print(f"  ✓ Cleaned: {len(cleaned_df):,} samples")

    # Step 4: Sample dataset
    print(f"\n[4/6] Sampling {SAMPLE_SIZE} samples...")
    sampled_df = sample_dataset(cleaned_df, n_samples=SAMPLE_SIZE)
    print(f"  ✓ Sampled: {len(sampled_df):,} samples")

    # Step 5: Load test datasets
    print("\n[5/6] Loading test datasets...")
    test_datasets_df = load_test_datasets()
    print(f"  ✓ Test datasets: {len(test_datasets_df):,} samples")

    # Step 6: Create splits
    print("\n[6/6] Creating train/validation/test splits...")
    train_df, val_df, test_df, combined_df = create_train_val_test_splits(sampled_df, test_datasets_df)
    print(f"  ✓ Train: {len(train_df):,} samples")
    print(f"  ✓ Validation: {len(val_df):,} samples")
    print(f"  ✓ Test: {len(test_df):,} samples")

    # Save outputs
    print("\nSaving outputs...")
    train_df.to_csv(train_output, index=False)
    val_df.to_csv(val_output, index=False)
    test_df.to_csv(test_output, index=False)
    combined_df.to_csv(combined_output, index=False)
    print(f"  ✓ Saved to {OUTPUT_DIR}/")

    print("\n" + "=" * 60)
    print("✓ Data preparation complete!")
    print("=" * 60)
else:
    print("\n✓ Data preparation outputs already exist.")
    print("  Skipping data preparation (using existing outputs)")


Checking if data preparation outputs exist...

Data preparation outputs not found. Running data preparation pipeline...
This may take a few minutes...

[1/6] Combining CSV files from Dataset/raw...
  ✓ Combined 251,665 raw samples

[2/6] Processing to master format...
  ✓ Master format: 249,893 samples

[3/6] Cleaning email text...


Cleaning Emails:   0%|          | 0/249893 [00:00<?, ?it/s]

  ✓ Cleaned: 249,586 samples

[4/6] Sampling 1000 samples...
  ✓ Sampled: 1,000 samples

[5/6] Loading test datasets...


Cleaning Test Emails:   0%|          | 0/242 [00:00<?, ?it/s]

  ✓ Test datasets: 242 samples

[6/6] Creating train/validation/test splits...
  ✓ Train: 869 samples
  ✓ Validation: 186 samples
  ✓ Test: 187 samples

Saving outputs...
  ✓ Saved to outputs/

✓ Data preparation complete!


## Precompute embeddings with a sentence transformer

In [5]:
!pip install -U -qq sentence-transformers

In [6]:
# load relevant libraries
import transformers
from transformers import BertModel, BertTokenizer, get_linear_schedule_with_warmup
from torch.optim import AdamW
import torch
from tqdm.auto import tqdm
from torch.utils.data import TensorDataset, DataLoader
from sentence_transformers import SentenceTransformer

In [7]:
# Global vars

BATCH_SIZE = 16
MAX_LEN = 400
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# precompute the embeddings

def precompute_embeddings_ST(model, data):
    all_embeddings = []
    model.to(device)

    for email_txt in tqdm(data, desc=f"Embedding emails"):
        _embedding = model.encode(email_txt, convert_to_tensor=True)
        all_embeddings.append(_embedding)

    # Concatenate into one big tensor
    # all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

In [9]:
# turn the df into dataloader
train_email_txt = train_df['text'].to_numpy()
train_label = train_df['label'].to_numpy()

# get the email emeddings
train_embeddings = precompute_embeddings_ST(model, train_email_txt)

Embedding emails:   0%|          | 0/869 [00:00<?, ?it/s]

In [10]:
# This cell is a smoke test I will find two emails with a high similarity score just to make sure
# the embedding process works

max_similarity = -1.0
most_similar_indices = (-1, -1)
similarity_threshold = 0.9


for i in range(len(train_embeddings)):
    for j in range(i + 1, len(train_embeddings)):
        # Calculate similarity between embedding i and embedding j
        similarity = model.similarity(train_embeddings[i], train_embeddings[j])

        current_similarity = similarity.item()

        if current_similarity > max_similarity:
            max_similarity = current_similarity
            most_similar_indices = (i, j)
        if max_similarity > similarity_threshold:
            break
    if max_similarity > similarity_threshold:
        break

email1_index, email2_index = most_similar_indices
email1_text = train_email_txt[email1_index]
email2_text = train_email_txt[email2_index]

print(f"The two most similar emails are at indices {email1_index} and {email2_index}.")
print(f"Similarity score: {max_similarity:.4f}")
print("\n--- Email 1 ---")
print(email1_text)
print("\n--- Email 2 ---")
print(email2_text)

The two most similar emails are at indices 28 and 101.
Similarity score: 0.9661

--- Email 1 ---
subject cisco inc student empowerment program apply nownndear studentnnwere pleased to offer you a parttime remote position through cisco incs student empowerment initiative this role requires only a few hours per week and pays a competitive weekly wagennto begin onboarding please reply with your alternate email address further instructions will be sent via private emailnnthis opportunity is limited to select students respond promptly to secure your spot

--- Email 2 ---
subject cisco systems student empowerment programnndear studentnnwe got your contact through your school database and are pleased to offer you a parttime remote position with cisco systems inc this role requires only a few hours per week and pays 350 weeklynnto begin onboarding please reply with your personal email address further instructions will be sent via private emailnnthis opportunity is part of our student empowerme

### Visualize the email embeddings

In [11]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import umap

In [90]:
import plotly.express as px

def plot_embeddings_3d(embeddings_3d, labels, title="3D Embedding Visualization", hover_txt = None, discrete_lbl = True):
    if discrete_lbl:
      labels_str = [str(label) for label in labels]
    else:
      labels_str = labels

    fig = px.scatter_3d(
        x=embeddings_3d[:, 0],
        y=embeddings_3d[:, 1],
        z=embeddings_3d[:, 2],
        color=labels_str,
        opacity=0.7,
        title=title,
        hover_name=hover_txt,
        # color_continuous_scale="bluered"
    )

    fig.update_layout(
        scene=dict(
            xaxis_title="Dim 1",
            yaxis_title="Dim 2",
            zaxis_title="Dim 3"
        ),
        legend_title="Labels"
    )

    fig.update_traces(
        marker=dict(size=5),
        selector=dict(mode='markers'))

    fig.show()

In [13]:
# Process the emdeddings
train_embeddings_np = [emb.cpu().numpy() for emb in train_embeddings]
train_embeddings_norm = normalize(train_embeddings_np, norm='l2')

In [14]:
# Doing PCA on the train embeddings
pca = PCA(n_components=3)
pca_reduced_embeddings = pca.fit_transform(train_embeddings_norm)

In [15]:
source_file_idx = [f for f in train_df['source_file'].unique()]
email_source_idx = [source_file_idx.index(f) for f in train_df['source_file']]

In [18]:
plot_embeddings_3d(embeddings_3d=pca_reduced_embeddings, labels=email_source_idx, hover_txt=train_df['source_file'], title="Emails Embeded with PCA by source")

In [17]:
plot_embeddings_3d(embeddings_3d=pca_reduced_embeddings, labels=train_label, hover_txt=train_df['source_file'], title="Emails Embeded with PCA by label")

In [91]:
def reduce_umap_3d(_embeddings):
  umap_reducer = umap.UMAP(
      n_components=3,
      metric='cosine',
      random_state=42
  )
  # Fit and transform embeddings
  return umap_reducer.fit_transform(_embeddings)

umap_reduced_embeddings = reduce_umap_3d(train_embeddings_norm)


/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [136]:
plot_embeddings_3d(embeddings_3d=umap_reduced_embeddings, labels=train_label, hover_txt=train_df['source_file'], title="Emails Embeded with UMAP")

## Cluster the email embeddings

In [117]:
from sklearn.cluster import KMeans
from sklearn.cluster import HDBSCAN
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.cluster import AgglomerativeClustering


In [20]:
def umap_reduce_embeddings(_embeddings):
  reducer = umap.UMAP(n_components=50, metric='cosine', random_state=42)
  reduced_embeddings = reducer.fit_transform(_embeddings)
  return reduced_embeddings

In [21]:
umap_reduced_train_embeddings = umap_reduce_embeddings(train_embeddings_norm)
distance_matrix = cosine_distances(umap_reduced_train_embeddings)
hdbscan = HDBSCAN(min_samples=5, metric="cosine")
cluster_labels = hdbscan.fit_predict(distance_matrix)

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



## Calcualte a cluster medoid and risk score

In [59]:
# A smoke test that will use the sentence transformer similarity function to see if the clustering we did is heading in the right direction
def get_similarities (embedded_data_df):
  embeddings_list = embedded_data_df['norm_emdedding'].to_list()
  similarities = model.similarity(embeddings_list, embeddings_list)
  return similarities

In [69]:
embedded_train_df = pd.DataFrame({
    'text': train_email_txt,
    'label': train_label,
    'norm_emdedding': list(train_embeddings_norm),
    'cluster_idx': cluster_labels,
    'source_file': train_df['source_file']
})
clusters_data = []

for cluster_idx, group in embedded_train_df.groupby('cluster_idx'):
    if cluster_idx == -1:
      continue
    # similarities = get_similarities(group)
    # print(f"Cluster {cluster_idx}, {len(group)} emails with similarity score: {similarities.mean().item():.2f}")
    cluster_embeddings = np.stack(group['norm_emdedding'].values)
    centroid = np.mean(cluster_embeddings, axis=0)
    distances = np.linalg.norm(cluster_embeddings - centroid, axis=1)
    medoid_idx = np.argmin(distances)
    medoid_text = group.iloc[medoid_idx]['text']
    medoid_label = group.iloc[medoid_idx]['label']
    medioid_embedding = group.iloc[medoid_idx]['norm_emdedding']
    medoid_source_file = group.iloc[medoid_idx]['source_file']
    risk_score = np.mean(group['label'])
    clusters_data.append({
        'cluster_idx': cluster_idx,
        'cluster_size': len(group),
        'centroid': centroid,
        'medoid_text': medoid_text,
        'medoid_label': medoid_label,
        'medioid_embedding': medioid_embedding,
        'medoid_source_file': medoid_source_file,
        'risk_score': risk_score
    })

clusters_df = pd.DataFrame(clusters_data)

In [88]:
# Cluster should have low similarity between each other. let's check

similarities = model.similarity(clusters_df['medioid_embedding'], clusters_df['medioid_embedding'])
similarities_fltrd = similarities[similarities != 1]
print(f"Mean cluster similarity is: {torch.mean(torch.abs(similarities_fltrd))}")

Mean cluster similarity is: 0.1929089790086452


In [93]:
cluster_3d_embeddings = reduce_umap_3d(np.stack(clusters_df['medioid_embedding'].values))
plot_embeddings_3d(embeddings_3d=cluster_3d_embeddings, labels=clusters_df['risk_score'], title='Clusters and their risk score', hover_txt=clusters_df['medoid_source_file'], discrete_lbl=False)

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



## Evaluation of validation and test Emails

In [129]:
# This function will take a df of emails, calculate their embedding, and normalize them. Then find the nearest cluster centroid and inherit it's score
def calculate_risk_scores(eval_emails_df):
  eval_embeddings = precompute_embeddings_ST(model, eval_emails_df['text'].to_numpy())
  eval_embeddings_np = [emb.cpu().numpy() for emb in eval_embeddings]

  eval_embeddings_norm = normalize(eval_embeddings_np, norm='l2')

  sims = cosine_similarity(eval_embeddings_norm, np.stack(clusters_df['medioid_embedding'].values))
  nearest_cluster_idx = sims.argmax(axis=1)

  eval_risk_scores = clusters_df.loc[nearest_cluster_idx, 'risk_score'].values
  return eval_risk_scores

In [138]:
def eval_accuracy(eval_risk_scores, eval_labels):
  eval_labels_np = eval_labels.to_numpy()
  squared_differences = np.square(eval_labels_np - eval_risk_scores)
  mse = np.mean(squared_differences)
  return (1 - mse)

In [145]:
# combine val and test df
eval_df = pd.concat([val_df, test_df], ignore_index=True)
eval_risk_scores = calculate_risk_scores(eval_df)
eval_accuracy = eval_accuracy(eval_risk_scores, eval_df['label'])

Embedding emails:   0%|          | 0/373 [00:00<?, ?it/s]

In [146]:
eval_accuracy

np.float64(0.7986912457111998)